In [ ]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt

# === Configuration ===
point_cloud_path = "your_point_cloud_file.ply"  # <- replace with your path
radius_values = [0.5, 1.0, 1.5, 2.0, 3.0]
max_nn_values = [15, 30]
bins = 180

# === Load point cloud ===
pcd = o3d.io.read_point_cloud(point_cloud_path)

# === Main loop ===
for radius in radius_values:
    for max_nn in max_nn_values:
        print(f"Estimating normals with radius={radius}, max_nn={max_nn}")

        # Copy the point cloud so we don't overwrite previous normals
        pcd_copy = pcd.voxel_down_sample(voxel_size=0.1)  # Optional downsample
        pcd_copy.estimate_normals(
            search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius, max_nn=max_nn),
            fast_normal_computation=True
        )

        normals = np.asarray(pcd_copy.normals)

        # Compute azimuth and zenith
        az = np.degrees(np.arctan2(normals[:, 1], normals[:, 0]))
        zen = np.degrees(np.arccos(np.clip(normals[:, 2], -1.0, 1.0)))

        # Plot histograms
        fig, axs = plt.subplots(1, 2, figsize=(10, 4))
        axs[0].hist(az, bins=bins, color='steelblue', edgecolor='black')
        axs[0].set_title(f"Azimuth (r={radius}, nn={max_nn})")
        axs[0].set_xlabel("Azimuth (degrees)")
        axs[1].hist(zen, bins=bins, color='tomato', edgecolor='black')
        axs[1].set_title(f"Zenith (r={radius}, nn={max_nn})")
        axs[1].set_xlabel("Zenith (degrees)")
        plt.suptitle(f"Normal Histogram (radius={radius}, max_nn={max_nn})")
        plt.tight_layout()
        plt.show()